In [29]:
import pandas as pd

In [79]:
!pwd

/Users/chunj/projects/cellranger-atac-mm9/tmp/test


In [30]:
%%time

df = pd.read_csv(
    "fragments.tsv.gz",
    sep="\t",
    header=None,
    names=["chr", "start", "end", "cb", "counts"]
)

CPU times: user 1min 9s, sys: 3.57 s, total: 1min 13s
Wall time: 1min 13s


In [31]:
df.shape

(106419711, 5)

In [32]:
df.head()

,chr,start,end,cb,counts
0,chr1,2999981,3000123,GAGTGAGTCGAGAAGC-1,1
1,chr1,2999983,3000189,TCTAGTTGTATCTGCA-1,1
2,chr1,3000505,3000665,AACGTACAGTGATAAC-1,1
3,chr1,3001165,3001324,TTAGGTGGTCTAAGAA-1,1
4,chr1,3001213,3001368,TGATTTCCAATGGTAA-1,1


In [33]:
df.dtypes

chr       object
start      int64
end        int64
cb        object
counts     int64
dtype: object

In [34]:
chrs = "chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chrX chrY chrM"
ends = "197195432 181748087 159599783 155630120 152537259 149517037 152524553 131738871 124076172 129993255 121843856 121257530 120284312 125194864 103494974 98319150 95272651 90772031 61342430 166650296 15902555 16299"

chrs = chrs.split(" ")
ends = ends.split(" ")

In [47]:
df_contigs = pd.DataFrame({"chromosome": chrs, "len": ends})

In [48]:
df_contigs.len = df_contigs.len.astype(int)

In [49]:
df_contigs

,chromosome,len
0,chr1,197195432
1,chr2,181748087
2,chr3,159599783
3,chr4,155630120
4,chr5,152537259
5,chr6,149517037
6,chr7,152524553
7,chr8,131738871
8,chr9,124076172
9,chr10,129993255


In [50]:
df_contigs.dtypes

chromosome    object
len            int64
dtype: object

In [77]:
%%time

problematics = []

with pysam.Tabixfile("fragments.tsv.gz") as tbx:
    for i, contig in df_contigs[ df_contigs.chromosome != "chrM" ].iterrows():

        for row in tbx.fetch(
            contig.chromosome,
            contig.len - 1,
            contig.len,
            parser=pysam.asBed()
        ):
            problematics.append(row)

CPU times: user 80.9 ms, sys: 4.54 ms, total: 85.4 ms
Wall time: 86 ms


In [ ]:
from frogress import bar

In [74]:
%%time

for p in bar(problematics):
    
    # retrieve the proper end of the contig
    proper_end = df_contigs.loc[ df_contigs.chromosome == p.contig, "len" ].values[0]
    
    # correct
    df.loc[ (df.chr == p.contig) & (df.start == p.start) & (df.cb == p.name), "end" ] = proper_end    

CPU times: user 5min 10s, sys: 23.6 s, total: 5min 34s
Wall time: 5min 34s


In [75]:
%%time

df.to_csv(
    "fragments_new.tsv",
    sep="\t",
    header=False,
    index=False
)

CPU times: user 6min 2s, sys: 11.7 s, total: 6min 14s
Wall time: 6min 19s
